<a href="https://colab.research.google.com/github/MateoRivera/network-dynamics/blob/main/functions_for_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Handle matrices and vectors
import numpy as np

# Handle Graphs
import networkx as nx

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Algebraic graph theory

In [105]:
def compute_P(W):
    # P = D^-1 @ W
    D = np.diag(W.sum(axis=1))
    return np.linalg.inv(D) @ W

def compute_L(W):
    # L = D - W
    D = np.diag(W.sum(axis=1))
    return D - W

def compute_pi(W, target_eigenvalue = 1, use_transition_matrix=True, decimal_precision=3):
    P = compute_P(W) if use_transition_matrix else W
    values, vectors = np.linalg.eig(P.T)
    target_eigenvalue = [i for i, eigenvalue in enumerate(values) if np.isclose(values[i], target_eigenvalue)]

    pi = vectors[:, target_eigenvalue].real
    for i in range(pi.shape[1]):
        pi[:, i] *= np.linalg.norm(pi[:, i])
        pi[:, i] /= np.sum(pi[:, i])

    round_vectorized = np.vectorize(lambda x: round(x, decimal_precision))
    pi = round_vectorized(pi)
    print("⚠️ Note: This result is an approximation due to potential numerical errors ⚠️")
    return pi

def compute_pi_bar(W, decimal_precision = 3):
    return compute_pi(compute_L(W), target_eigenvalue=0, use_transition_matrix=False, decimal_precision=decimal_precision)

## Averaging models with stubborn nodes

In [137]:
def compute_equilibria_with_stubborn(W, stubborn_id, u):
    u = np.array(u)
    if len(u.shape) == 1: # Is not a column vector
        u = u.reshape(-1,1)
    P = compute_P(W)
    regular_id = list(filter(lambda x: x not in stubborn_id, range(len(W))))
    Q = P[np.ix_(regular_id, regular_id)]
    E = P[np.ix_(regular_id, stubborn_id)]
    I = np.eye(len(Q))

    return np.linalg.inv((I-Q)) @ (E @ u)

## Examples of use

In [106]:
W = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0]
])

compute_pi(W)

⚠️ Note: This result is an approximation due to potential numerical errors ⚠️


array([[ 0.333,  0.   ],
       [ 0.333,  0.   ],
       [ 0.333,  0.   ],
       [-0.   ,  0.   ],
       [-0.   , -0.   ],
       [-0.   ,  0.   ],
       [-0.   ,  0.2  ],
       [-0.   ,  0.4  ],
       [-0.   ,  0.4  ]])

In [112]:
compute_pi_bar(W, 5)

⚠️ Note: This result is an approximation due to potential numerical errors ⚠️


array([[ 0.33333,  0.01351],
       [ 0.33333,  0.01351],
       [ 0.33333,  0.01351],
       [ 0.     , -0.     ],
       [ 0.     ,  0.     ],
       [ 0.     , -0.     ],
       [ 0.     ,  0.23987],
       [ 0.     ,  0.23987],
       [ 0.     ,  0.47973]])

In [138]:
compute_equilibria_with_stubborn(W, [0,8], np.array([[0], [1]]))

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.36363636],
       [0.45454545],
       [1.        ],
       [1.        ]])

In [139]:
compute_equilibria_with_stubborn(W, [0,8], [0,1])

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.36363636],
       [0.45454545],
       [1.        ],
       [1.        ]])